In [2]:
import numpy as np
import pandas as pd
import json
import os

pd.set_option('display.max_columns',None)

/FileStore/tables/estados.csv


In [3]:
read = open('./metadata.json','r')
metadata = json.loads(read.read())
read.close()

In [4]:
metadata.keys()

dict_keys(['doc_desc', 'study_desc'])

# limpieza de datos


In [4]:
df = pd.read_csv('gastoshogar.txt'
                 ,dtype={x:'str' for x in ['clave', 'forma_pag1', 'forma_pag2', 'forma_pag3', 'frecuencia', 'inst_1', 'inst_2', 'lugar_comp', 'orga_inst', 'tipo_gasto', 'mes_dia', 'fecha_adqu', 'fecha_pago', 'folioviv', 'foliohog']}
                 ,na_values=['0000','00','0'] 
                 ,quotechar="'"
                 )

C:\Users\brenda.linares\AppData\Local\Temp\ipykernel_22608\3629872917.py:1: DtypeWarning: Columns (13,14,16,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('gastoshogar.txt'


In [5]:
# El siguiente es un ejemplo de la respuesta esperada, sin embargo, para conseguir el punto debe encontrar el error en la función

def valores_numericos(data:pd.DataFrame, columnas:list):
    """Recibe un DataFrame y un listado de columnas y transforma el tipo de dato de estas ultimas a flotante"""
    data_out = data.copy() #recuerda hacer un copy de la tabla a manipular en la función para evitar problemas de relación
    for columna in columnas:
        data_out[columna] = pd.to_numeric(data_out[columna],errors='coerce').astype(float)
    return data_out

# El siguiente es un ejemplo de la respuesta esperada, sin embargo, para conseguir el punto debe encontrar el error en la función
def no_nulo(data:pd.DataFrame, min_pct_no_nulo:float=0.60):
    """Recibe un DataFrame y un listado de columnas y transforma el tipo de dato de estas ultimas a flotante"""
    data_out = data.copy() #recuerda hacer un copy de la tabla a manipular en la función para evitar problemas de relación
    drop_cols = []
    
    for column in range(len(data_out.columns)):
        if (data_out.iloc[:,column].isnull().sum() / len(data_out.iloc[:,column]) ) > min_pct_no_nulo:
            drop_cols.append(data_out.columns[column])
            
        else:
            pass
    
    data_out = data_out.drop(axis=1,labels=drop_cols)
    return data_out

def no_unarias(data:pd.DataFrame, columnas:list, max_pct_repeticion:float=0.90):
    data_out = data.copy() #recuerda hacer un copy de la tabla a manipular en la función para evitar problemas de relación
    drop_cols = []

    for columna in columnas:
        if columna in data_out.columns:
            if (data_out[columna].value_counts().max() / len(data_out.iloc[:,1]) ) > max_pct_repeticion:
                drop_cols.append(columna)
            else:
                pass
        else:
            pass
    
    data_out = data_out.drop(axis=1,labels=drop_cols)
    return data_out

In [6]:
def limpieza_de_datos(data:pd.DataFrame, columnas_numericas:list, columnas_no_numericas:list, min_pct_no_nulo:float=0.60, max_pct_repeticion:float=0.90):
    data_out = data.copy() #recuerda hacer un copy de la tabla a manipular en la función para evitar problemas de relación
    
    num_vals = valores_numericos(data_out,columnas_numericas)
    print('numeric values are now FLOATS')
    
    sin_null = no_nulo(num_vals, min_pct_no_nulo=min_pct_no_nulo)
    print('null values removed')
    
    
    for c in columnas_no_numericas:
        if c in sin_null.columns:
            data_out = no_unarias(sin_null, columnas_no_numericas, max_pct_repeticion=max_pct_repeticion)
        else:
            pass
    print('var disccretas limpias')
    
    return data_out

# Hint: algunas columnas no numericas serán eliminadas por la función de no_nulo, por lo que al querer borrarlas en no_unarias puede producir que no encuentre la columna

In [9]:
cols_num = ['cantidad', 'gasto', 'pago_mp', 'costo', 'inmujer', 'num_meses', 'num_pagos', 'gasto_tri', 'gasto_nm', 'gas_nm_tri', 'imujer_tri']
cols_nonum = ['clave', 'tipo_gasto', 'mes_dia', 'forma_pag1','forma_pag2', 'forma_pag3', 'lugar_comp', 'orga_inst', 'frecuencia','fecha_adqu', 'fecha_pago', 'inst_1', 'inst_2', 'ultim_pago']

In [8]:
df_f = limpieza_de_datos(
    data=df
    ,columnas_numericas=cols_num
    ,columnas_no_numericas=cols_nonum
    ,min_pct_no_nulo=0.7
    ,max_pct_repeticion=0.9
    )
df_f.head()

numeric values are now FLOATS
null values removed
var disccretas limpias


,folioviv,foliohog,clave,mes_dia,forma_pag1,lugar_comp,cantidad,gasto,gasto_tri
0,0100013606,1,C001,NaN,01,06,NaN,100.0,300.0
1,0100013606,1,C003,NaN,01,06,NaN,40.0,120.0
2,0100013606,1,C004,NaN,01,06,NaN,44.0,132.0
3,0100013606,1,C005,NaN,01,06,NaN,46.0,138.0
4,0100013606,1,C006,NaN,01,06,NaN,32.0,96.0


In [12]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1409409 entries, 0 to 1409408
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   folioviv    1409409 non-null  object 
 1   foliohog    1409409 non-null  object 
 2   clave       1409409 non-null  object 
 3   mes_dia     705526 non-null   object 
 4   forma_pag1  1261869 non-null  object 
 5   lugar_comp  1135288 non-null  object 
 6   cantidad    705526 non-null   float64
 7   gasto       1280294 non-null  float64
 8   gasto_tri   1278856 non-null  float64
dtypes: float64(3), object(6)
memory usage: 96.8+ MB


In [10]:
columnas_numericas = [x for x in cols_num if x in df_f.columns]
columnas_no_numericas = [x for x in cols_nonum if x in df_f.columns]

# Cruce tablas


In [13]:
estados_df = pd.read_csv('estados.csv'
                 ,na_values=['0000','00','0'] 
                 ,quotechar="'"
                 )

In [14]:
estados_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   clave     34 non-null     int64 
 1   Enitidad  34 non-null     object
dtypes: int64(1), object(1)
memory usage: 672.0+ bytes


In [11]:
catalogos = os.listdir('./catalogos/')

In [12]:
cat = dict()
for x in catalogos:
    read = open(f'./catalogos/{x}','r')
    json_r = json.loads(read.read())
    read.close()
    json_r = pd.DataFrame(json_r['var_catgry'])
    name = x.split('.')[0]
    json_r.rename(columns={'value':name,'labl':f'{name} codificación'},inplace=True)
    cat[name] = json_r[[name,f'{name} codificación']]

In [13]:
cat.keys()

dict_keys(['clave', 'forma_pag1', 'forma_pag2', 'forma_pag3', 'frecuencia', 'inst_1', 'inst_2', 'lugar_comp', 'orga_inst', 'tipo_gasto'])

In [15]:
for col in columnas_no_numericas:
    try:
        df_f = df_f.merge(cat[col],how='left')
    except:
        None

In [16]:
df_f.head(5)

,folioviv,foliohog,clave,mes_dia,forma_pag1,lugar_comp,cantidad,gasto,gasto_tri,clave codificación,forma_pag1 codificación,lugar_comp codificación
0,0100013606,1,C001,NaN,01,06,NaN,100.0,300.0,"Detergentes (polvo, líquido, pasta, gel)",Efectivo,Supermercados
1,0100013606,1,C003,NaN,01,06,NaN,40.0,120.0,Blanqueadores,Efectivo,Supermercados
2,0100013606,1,C004,NaN,01,06,NaN,44.0,132.0,Suavizantes de telas,Efectivo,Supermercados
3,0100013606,1,C005,NaN,01,06,NaN,46.0,138.0,Limpiadores (en polvo o líquido),Efectivo,Supermercados
4,0100013606,1,C006,NaN,01,06,NaN,32.0,96.0,Servilletas y papel absorbente,Efectivo,Supermercados


In [21]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1409409 entries, 0 to 1409408
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   folioviv    1409409 non-null  object 
 1   foliohog    1409409 non-null  object 
 2   clave       1409409 non-null  object 
 3   mes_dia     705526 non-null   object 
 4   forma_pag1  1261869 non-null  object 
 5   lugar_comp  1135288 non-null  object 
 6   cantidad    705526 non-null   float64
 7   gasto       1280294 non-null  float64
 8   gasto_tri   1278856 non-null  float64
dtypes: float64(3), object(6)
memory usage: 96.8+ MB


In [20]:
clave_df = cat['clave']
forma_pag1_df = cat['forma_pag1']
lugar_comp_df = cat['lugar_comp']

clave_df.to_csv("catalogos-csv/clave.csv", index=False)
forma_pag1_df.to_csv("catalogos-csv/forma_pag1.csv", index=False)
lugar_comp_df.to_csv("catalogos-csv/lugar_comp.csv", index=False)